**Introduction**
From the meta data generate shape files and upload them to google earth engine as feature collections.And for each feature collection run the codes in the notebook.First 2 cells are for shapefiles generation

In [ ]:
import geopandas as gpd
from shapely.geometry import shape
import pandas as pd
import numpy as np
df_ofp=pd.read_csv("/content/drive/MyDrive/concatenated_df_Asia.csv")
df_ofp['host_name'] = 'Verra'
df_ofp["site_id_reported"]=['asi_reforest_site_{}'.format(i+1) for i in range(len(df_ofp))]
df_ofp["project_id_reported"]="Verra_Asia_1"
df_ofp['url'] = 'https://registry.verra.org/app/projectDetail/CCB/2374'
df_ofp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43628 entries, 0 to 43627
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   site_id_reported          43628 non-null  object 
 1   project_id_reported       43628 non-null  object 
 2   geometry_reported         43611 non-null  object 
 3   description_reported      0 non-null      float64
 4   planting_date_reported    0 non-null      float64
 5   trees_planted_reported    0 non-null      float64
 6   survival_rate_reported    0 non-null      float64
 7   species_count_reported    0 non-null      float64
 8   species_planted_reported  0 non-null      float64
 9   country                   43610 non-null  object 
 10  site_sqkm                 43628 non-null  float64
 11  host_name                 43628 non-null  object 
 12  url                       43628 non-null  object 
dtypes: float64(7), object(6)
memory usage: 4.3+ MB


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import mapping

def load_wkt(wkt_string):
    try:
        return wkt.loads(wkt_string)
    except TypeError:
        return wkt_string


df_ofp['geometry_reported'] = df_ofp['geometry_reported'].apply(load_wkt)


gdf = gpd.GeoDataFrame(df_ofp, geometry='geometry_reported')


gdf['geometry_reported'] = gdf['geometry_reported'].apply(lambda geom: geom if geom is not None and geom.is_valid else geom.buffer(0) if geom is not None else None)


gdf['geometry_reported'] = gdf['geometry_reported'].simplify(tolerance=0.001, preserve_topology=True)


def validate_and_correct_geometry(geom):
    if geom is None:
        return None
    if not geom.is_valid:
        geom = geom.buffer(0)

    return geom

gdf['geometry_reported'] = gdf['geometry_reported'].apply(validate_and_correct_geometry)


gdf.set_crs("EPSG:4326", inplace=True)

#saving the shape file
gdf.to_file("/content/drive/MyDrive/verra_asia.shp") # upload the shapefile to earth engine as an assset then perform all the other columns extractions

<ipython-input-6-e410c0f5fe33>:39: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("/content/drive/MyDrive/verra_asia.shp") # upload the shapefile to earth engine as an assset then perform all the other columns extractions


In [ ]:
import ee
from google.colab import auth

auth.authenticate_user()
ee.Initialize(project='protean-keyword-421212')


##TREE COVER COLUMNS; "tree_cover_area_2000,tree_cover_area_2005 tree_cover_area_2010,tree_cover_area_2015 AND tree_cover_area_2020

In [ ]:
#Loading the Polygon as a feature collection from earth engine
plantp = ee.FeatureCollection('projects/ee-forest-monitoring/assets/plant_pp_new_shapefile_new')


landmask = ee.Image("projects/glad/OceanMask").lte(1)
landCover = ee.Image('projects/glad/GLCLU2020/v2/LCLUC_2020').updateMask(landmask) # CHANGE YEARS 2000,2005,2010,2015,2020


classCodes = [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
              125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148]

# Mask the land cover image
maskedLandCover = landCover.remap(classCodes, classCodes, 255)

# Function to calculate area for each class within each polygon
def calculate_area(feature):
    total_area = ee.Number(0)

    def calculate_class_area(class_code, total):
        class_area = maskedLandCover.eq(ee.Number(class_code)) \
            .multiply(ee.Image.pixelArea()) \
            .reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=feature.geometry(),
                scale=30,
                maxPixels=1e9
            ).get('remapped')

        area_value = ee.Number(class_area).divide(1e6)  # Convert to km^2
        return ee.Number(total).add(area_value)

    total_area = ee.List(classCodes).iterate(calculate_class_area, total_area)
    return feature.set('tree_cover_area_2020', total_area)


area_results = plantp.map(calculate_area)


task = ee.batch.Export.table.toDrive(
    collection=area_results,
    description='asia',
    folder='Plant_planet',
    fileNamePrefix='new_2020_plant_land_cover_areas_with_total',
    fileFormat='CSV'
)
task.start()

print("Exporting...")


import time

while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(30)

print('Done with export.')




Exporting...
Polling for task (id: SQUT3THVV6ATZZX6TYATQLUD).
Polling for task (id: SQUT3THVV6ATZZX6TYATQLUD).
Polling for task (id: SQUT3THVV6ATZZX6TYATQLUD).
Polling for task (id: SQUT3THVV6ATZZX6TYATQLUD).
Done with export.


##CHANGE COLUMNS EXTRACTION ("permanent_water", "short_vegetation_after_tree_loss", "cropland_loss_to_tree", "cropland_gain_from_trees")

In [ ]:

plantp = ee.FeatureCollection('projects/ee-forest-monitoring/assets/plant_pp_new_shapefile_new')


landmask = ee.Image("projects/glad/OceanMask").lte(1)
landCover = ee.Image('projects/glad/GLCLU2020/v2/LCLUC').updateMask(landmask)


classCodes = [208, 240, 248, 245]
classesOfInterest = ["permanent_water", "short_vegetation_after_tree_loss", "cropland_loss_to_tree", "cropland_gain_from_trees"]

maskedLandCover = landCover.remap(classCodes, classCodes, 255)


def calculate_area(feature):

    areas = {}


    for class_code, class_name in zip(classCodes, classesOfInterest):
        current_class_area = maskedLandCover.eq(class_code) \
            .multiply(ee.Image.pixelArea()) \
            .reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=feature.geometry(),
                scale=30,
                maxPixels=1e9
            )

        area_value = ee.Number(current_class_area.get('remapped', 0)).divide(1e6)

        areas[class_name] = area_value


    return feature.set(areas)


area_results = plantp.map(calculate_area)


print('Area Results:', area_results.first().getInfo())


task = ee.batch.Export.table.toDrive(
    collection=area_results,
    description='atlas_land_cover_areas_2020',
    folder='Plant_planet',
    fileNamePrefix='plant_land_cover_areas_with_total',
    fileFormat='CSV'
)
task.start()

print('Exporting to Google Drive...')


import time

while task.active():
    print('Waiting for task to complete...')
    time.sleep(30)

print('Export completed.')


Area Results: {'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[145.95097037392966, -17.554767364379934], [145.95254886595544, -17.554994824668505], [145.95254886595544, -17.554874418234903], [145.95239725338658, -17.554789699202793], [145.95225456443393, -17.554722804996555], [145.9519379980211, -17.55472730156947], [145.95184434556455, -17.554812005072314], [145.9517284274125, -17.554847641666935], [145.95152778057894, -17.55482089599683], [145.95143412854463, -17.554767376229083], [145.95125574803092, -17.554745114128277], [145.951171019534, -17.554620253507697], [145.9510015627769, -17.55451322874444], [145.95097037392966, -17.554767364379934]]]}, 'id': '000000000000000001a5', 'properties': {'country': 'AU', 'cropland_gain_from_trees': 0, 'cropland_loss_to_tree': 0, 'descriptio': 'Pin Gin Hill will have 5000 native trees planted to enhance and extend the vegetation along a tributary that flows into the Great Barrier Reef.\r\nRevegetation will support and enhance

##BUILT AREA CHARACTERISTICS EXTRACTION(BUILT_AREA)

In [ ]:


# Loading the Settlement Characteristics layer for 2018
builtImage = ee.Image("JRC/GHSL/P2023A/GHS_BUILT_C/2018").select('built_characteristics')


plantp = ee.FeatureCollection('projects/ee-forest-monitoring/assets/plant_pp_new_shapefile_new')

# Defining the number of patches due to the size of some polygon collections that causes computational timeout
patch_size = 10000
total_polygons = plantp.size()
num_patches = total_polygons.divide(patch_size).ceil()


def calculate_built_area(feature):
    polygon = feature.geometry()
    built_area = builtImage.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=polygon,
        scale=10,
        maxPixels=1e9
    ).get('built_characteristics')

    built_area = ee.Number(built_area).max(0)
    return feature.set('built_area', built_area)

def process_patch(patch_index):
    start = ee.Number(patch_index).multiply(patch_size)
    end = start.add(patch_size)

    patch = plantp.toList(patch_size, start)
    patch_fc = ee.FeatureCollection(patch)

    processed_patch = patch_fc.map(calculate_built_area)
    return processed_patch


patch_indices = ee.List.sequence(0, num_patches.subtract(1))
patches = patch_indices.map(process_patch)


processed_polygons = ee.FeatureCollection(patches).flatten()

first_feature = processed_polygons.first().getInfo()
print('First feature with built area:', first_feature)

# Exporting and saving  the results as csv to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=processed_polygons,
    folder='Plant_planet',
    description='New_plant_BuiltAreaStats_2018_PATCHED',
    fileFormat='CSV'
)
task.start()

print("Export started...")

# Monitoring the export task status
while task.active():
    print(f'Polling for task (id: {task.id})...')
    time.sleep(30)

print('Export completed.')


First feature with built area: {'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[145.95097037392966, -17.554767364379934], [145.95254886595544, -17.554994824668505], [145.95254886595544, -17.554874418234903], [145.95239725338658, -17.554789699202793], [145.95225456443393, -17.554722804996555], [145.9519379980211, -17.55472730156947], [145.95184434556455, -17.554812005072314], [145.9517284274125, -17.554847641666935], [145.95152778057894, -17.55482089599683], [145.95143412854463, -17.554767376229083], [145.95125574803092, -17.554745114128277], [145.951171019534, -17.554620253507697], [145.9510015627769, -17.55451322874444], [145.95097037392966, -17.554767364379934]]]}, 'id': '0_000000000000000001a5', 'properties': {'built_area': 0, 'country': 'AU', 'descriptio': 'Pin Gin Hill will have 5000 native trees planted to enhance and extend the vegetation along a tributary that flows into the Great Barrier Reef.\r\nRevegetation will support and enhance habitat for vulnerable

##ROAD NETWORKS COLUMNS(total_road_length_km)

In [ ]:

import time

In [ ]:


polygons = ee.FeatureCollection('projects/ee-forest-monitoring/assets/plant_pp_new_shapefile_new')

# Loading all road NASA feature collections
roadsAfrica = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-africa')
roadsAmericas = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-americas')
roadsAsia = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-asia')
roadsEurope = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-europe')
roadsOceaniaEast = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-oceania-east')
roadsOceaniaWest = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-oceania-west')

# Merging all road feature collections
roads = roadsAfrica.merge(roadsAmericas).merge(roadsAsia).merge(roadsEurope).merge(roadsOceaniaEast).merge(roadsOceaniaWest)

def calculate_road_length(polygon):
    # Intersecting roads with the polygon area
    intersecting_roads = roads.filterBounds(polygon.geometry())

    # Clipping the intersecting roads to the polygon geometry
    def clip_and_calculate_length(road):
        road_geom = road.geometry()
        polygon_geom = polygon.geometry()
        clipped = road_geom.intersection(polygon_geom, ee.ErrorMargin(1))
        return ee.Feature(clipped).set('length', clipped.length())

    clipped_roads = intersecting_roads.map(clip_and_calculate_length)

    # Calculating the total length of the clipped road segments
    road_length_sum = clipped_roads.reduceColumns(
        reducer=ee.Reducer.sum(),
        selectors=['length']
    ).get('sum')

    # Counting the number of intersecting roads
    intersecting_roads_count = intersecting_roads.size()

    # Setting properties for the polygon feature
    return polygon.set({
        'total_road_length_km': ee.Number(road_length_sum).divide(1000),  # Convert to kilometers
        'intersecting_roads_count': intersecting_roads_count
    })

# Map the function over the Feature Collection
polygons_with_road_length = polygons.map(calculate_road_length)


first_feature = polygons_with_road_length.first().getInfo()
print('First feature with road length and count:', first_feature)
# Exporting and saving  the results as csv to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=polygons_with_road_length,
    folder='Plant_planet',
    description='new_plant_with_road_length',
    fileFormat='CSV'
)
task.start()

print("Export started...")


while task.active():
    print(f'Polling for task (id: {task.id})...')
    time.sleep(30)

print('Export completed.')


First feature with road length and count: {'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[145.95097037392966, -17.554767364379934], [145.95254886595544, -17.554994824668505], [145.95254886595544, -17.554874418234903], [145.95239725338658, -17.554789699202793], [145.95225456443393, -17.554722804996555], [145.9519379980211, -17.55472730156947], [145.95184434556455, -17.554812005072314], [145.9517284274125, -17.554847641666935], [145.95152778057894, -17.55482089599683], [145.95143412854463, -17.554767376229083], [145.95125574803092, -17.554745114128277], [145.951171019534, -17.554620253507697], [145.9510015627769, -17.55451322874444], [145.95097037392966, -17.554767364379934]]]}, 'id': '000000000000000001a5', 'properties': {'country': 'AU', 'descriptio': 'Pin Gin Hill will have 5000 native trees planted to enhance and extend the vegetation along a tributary that flows into the Great Barrier Reef.\r\nRevegetation will support and enhance habitat for vulnerable Possum 

##GLOBAL FOREST LOSS GROUPS EXTRACTION,using the python code in the "data_analysis.ipny" create the colun s "loss_pre_5","loss_post_3",loss_post_5"

In [ ]:

plantp = ee.FeatureCollection('projects/ee-forest-monitoring/assets/plant_pp_new_shapefile_new')
gfc2017 = ee.Image('UMD/hansen/global_forest_change_2023_v1_11')



#Defining a Function to calculate forest loss for each polygon
def calculate_loss(feature):
    loss_image = gfc2017.select(['loss'])
    loss_area_image = loss_image.multiply(ee.Image.pixelArea())
    loss_year = gfc2017.select(['lossyear'])


    loss_by_year = loss_area_image.addBands(loss_year).reduceRegion(
        reducer=ee.Reducer.sum().group(groupField=1),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e9
    )

    # Setting the loss data as properties of the feature
    return feature.set(loss_by_year)


loss_results = plantp.map(calculate_loss)

first_result = loss_results.first().getInfo()
print(first_result)

# Exporting and saving the results as CSV file to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=loss_results,
    folder='Plant_planet',
    description='plant_forest_loss_by_polygon',
    fileFormat='CSV'
)
task.start()

print("Export started...")


while task.active():
    print(f'Polling for task (id: {task.id})...')
    time.sleep(30)

print('Export completed.')


{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[145.95097037392966, -17.554767364379934], [145.95254886595544, -17.554994824668505], [145.95254886595544, -17.554874418234903], [145.95239725338658, -17.554789699202793], [145.95225456443393, -17.554722804996555], [145.9519379980211, -17.55472730156947], [145.95184434556455, -17.554812005072314], [145.9517284274125, -17.554847641666935], [145.95152778057894, -17.55482089599683], [145.95143412854463, -17.554767376229083], [145.95125574803092, -17.554745114128277], [145.951171019534, -17.554620253507697], [145.9510015627769, -17.55451322874444], [145.95097037392966, -17.554767364379934]]]}, 'id': '000000000000000001a5', 'properties': {'country': 'AU', 'descriptio': 'Pin Gin Hill will have 5000 native trees planted to enhance and extend the vegetation along a tributary that flows into the Great Barrier Reef.\r\nRevegetation will support and enhance habitat for vulnerable Possum and Glider species.\r\nThe project will al

####Generating the loss columns

In [ ]:
import pandas as pd
df_km=pd.read_csv("/content/drive/MyDrive/Plant_planet/plant_forest_loss_by_polygon.csv")

df_km["groups"]

,groups
0,[]
1,[]
2,[]
3,[]
4,[]
...,...
1071,"[{group=15, sum=29764.3157698089}, {group=16, ..."
1072,"[{group=7, sum=5644.555928548177}, {group=10, ..."
1073,"[{group=3, sum=5151.752848307291}, {group=6, s..."
1074,"[{group=15, sum=793.3899449965534}, {group=16,..."


In [ ]:
df_km.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076 entries, 0 to 1075
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   system:index  1076 non-null   object 
 1   country       1076 non-null   object 
 2   descriptio    1076 non-null   object 
 3   groups        1076 non-null   object 
 4   planting_d    873 non-null    object 
 5   project_id    1076 non-null   object 
 6   site_id_re    1076 non-null   object 
 7   site_sqkm     1076 non-null   float64
 8   survival_r    887 non-null    float64
 9   trees_plan    1076 non-null   float64
 10  .geo          1076 non-null   object 
dtypes: float64(3), object(8)
memory usage: 92.6+ KB


In [ ]:
import ast


def parse_loss_groups(loss_groups):

    all_groups = {f"group_{i}": 0 for i in range(24)}  # defining the 24 groups (0 to 23)-2000 to 2023

    if not loss_groups or loss_groups == '[]':
        return all_groups

    loss_groups = loss_groups.replace('group=', '"group":').replace('sum=', '"sum":')
    parsed_groups = ast.literal_eval(loss_groups)


    for item in parsed_groups:
        all_groups[f"group_{item['group']}"] = item['sum']

    return all_groups


expanded_columns = df_km['groups'].apply(parse_loss_groups)


expanded_df = pd.DataFrame(expanded_columns.tolist())


group_mapping = {
    0: '2000',
    1: '2001',
    2: '2002',
    3: '2003',
    4: '2004',
    5: '2005',
    6: '2006',
    7: '2007',
    8: '2008',
    9: '2009',
    10: '2010',
    11: '2011',
    12: '2012',
    13: '2013',
    14: '2014',
    15: '2015',
    16: '2016',
    17: '2017',
    18: '2018',
    19: '2019',
    20: '2020',
    21: '2021',
    22: '2022',
    23: '2023'
}


expanded_df.rename(columns=lambda x: group_mapping.get(int(x.split('_')[1]), x), inplace=True)


df_km = df_km.join(expanded_df)

In [ ]:
#  When only year is given in planting_date
df_km["planting_date_reported"]=df_km["planting_d"]
# def calculate_loss_pre_5(row):
#     planted_year = row['planting_date_reported']
#     years = [str(planted_year - i) for i in range(1, 6)]
#     losses = [row[year] for year in years if year in df_km.columns]
#     return np.nanmean(losses) if losses else np.nan

# def calculate_loss_post_3(row):
#     planted_year = row['planting_date_reported']
#     years = [str(planted_year + i) for i in range(1, 4)]
#     losses = [row[year] for year in years if year in df_km.columns]
#     return np.nanmean(losses) if losses else np.nan
# def calculate_loss_post_5(row):
#     planted_year = row['planting_date_reported']
#     years = [str(planted_year + i) for i in range(1, 6)]
#     losses = [row[year] for year in years if year in df_km.columns]
#     return np.nanmean(losses) if losses else np.nan

# df_km['loss_pre_5'] = df_km.apply(calculate_loss_pre_5, axis=1)
# df_km['loss_post_3'] = df_km.apply(calculate_loss_post_3, axis=1)
# df_km['loss_post_5'] = df_km.apply(calculate_loss_post_5, axis=1)



# df_km[['planting_date_reported', 'loss_pre_5','loss_post_3']].tail()

In [ ]:
# if year not given and have to extract
import numpy as np

def calculate_loss_pre_5(row):
    try:
        planted_date = pd.to_datetime(row['planting_date_reported'], errors='coerce')
        if pd.isnull(planted_date):
            return np.nan
        planted_year = planted_date.year
        years = [str(planted_year - i) for i in range(1, 6)]
        losses = [row[year] for year in years if year in df_km.columns]
        return np.nanmean(losses) if losses else np.nan
    except Exception as e:
        return np.nan

def calculate_loss_post_3(row):
    try:
        planted_date = pd.to_datetime(row['planting_date_reported'], errors='coerce')
        if pd.isnull(planted_date):
            return np.nan
        planted_year = planted_date.year
        years = [str(planted_year + i) for i in range(1, 4)]
        losses = [row[year] for year in years if year in df_km.columns]
        return np.nanmean(losses) if losses else np.nan
    except Exception as e:
        return np.nan

def calculate_loss_post_5(row):
    try:
        planted_date = pd.to_datetime(row['planting_date_reported'], errors='coerce')
        if pd.isnull(planted_date):
            return np.nan
        planted_year = planted_date.year
        years = [str(planted_year + i) for i in range(1, 6)]
        losses = [row[year] for year in years if year in df_km.columns]
        return np.nanmean(losses) if losses else np.nan
    except Exception as e:
        return np.nan


df_km['loss_pre_5'] = df_km.apply(calculate_loss_pre_5, axis=1)
df_km['loss_post_3'] = df_km.apply(calculate_loss_post_3, axis=1)
df_km['loss_post_5'] = df_km.apply(calculate_loss_post_5, axis=1)

In [ ]:
columns_to_drop = [str(year) for year in range(2000, 2024)] + ['groups']
df_km = df_km.drop(columns=columns_to_drop, errors='ignore')
df_km.tail()

,system:index,country,descriptio,planting_d,project_id,site_id_re,site_sqkm,survival_r,trees_plan,.geo,planting_date_reported,loss_pre_5,loss_post_3,loss_post_5
1071,000000000000000000bc,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""Polygon"",""coordinates"":[[[38.24315018...",NaN,NaN,NaN,NaN
1072,000000000000000000bd,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""Polygon"",""coordinates"":[[[38.24283809...",NaN,NaN,NaN,NaN
1073,000000000000000000be,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""Polygon"",""coordinates"":[[[38.25306721...",NaN,NaN,NaN,NaN
1074,000000000000000000bf,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""Polygon"",""coordinates"":[[[38.24170103...",NaN,NaN,NaN,NaN
1075,000000000000000000c0,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""Polygon"",""coordinates"":[[[38.37896101...",NaN,NaN,NaN,NaN


In [ ]:
df_km.to_csv("/content/drive/MyDrive/Plant_planet/plant_forest_loss_by_polygon.csv")

##NDVI per MONTH

In [ ]:

# Gives each month separate file

geometry = ee.FeatureCollection("projects/ee-forest-monitoring/assets/plant_pp_new_shapefile_new")


geometry = geometry.map(lambda feature: feature.simplify(30))

# Loading the Sentinel-2 Image Collection for the year 2023
S2 = ee.ImageCollection('COPERNICUS/S2') \
  .filterDate('2023-01-01', '2023-12-31') \
  .filterBounds(geometry)


def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)


def reduce_to_monthly(image_collection):
    monthly = image_collection.select('NDVI').mean().rename('NDVI')
    return monthly


def calculate_and_export_monthly_ndvi(month):

    monthly_s2 = S2.filter(ee.Filter.calendarRange(month, month, 'month')).map(calculate_ndvi)


    monthly_ndvi = reduce_to_monthly(monthly_s2)

    mean_ndvi = monthly_ndvi.reduceRegions(
        collection=geometry,
        reducer=ee.Reducer.mean(),
        scale=30
    )

    # Exporting and saving the results to Google Drive
    task = ee.batch.Export.table.toDrive(
        collection=mean_ndvi,
        folder='Plant_planet',
        description=f'monthly_plant_{month}',
        fileFormat='CSV'
    )
    task.start()

# Looping through each month and process the data
for month in range(1, 13):
    calculate_and_export_monthly_ndvi(month)
    print(f'Exporting month {month}...')

tasks = ee.batch.Task.list()
while any([task.active() for task in tasks]):
    print('Waiting for all tasks to complete...')
    time.sleep(30)

print('All tasks completed.')


/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


Exporting month 1...
Exporting month 2...
Exporting month 3...
Exporting month 4...
Exporting month 5...
Exporting month 6...
Exporting month 7...
Exporting month 8...
Exporting month 9...
Exporting month 10...
Exporting month 11...
Exporting month 12...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiting for all tasks to complete...
Waiti

In [ ]:
import pandas as pd
import os


data_dir = '/content/drive/MyDrive/Plant_planet'


combined_df = pd.DataFrame()


for month in range(1, 13):

    file_path = os.path.join(data_dir, f'monthly_plant_{month}.csv')

    df = pd.read_csv(file_path)


    ndvi_column_name = f'NDVI_0{month}'
    df.rename(columns={'mean': ndvi_column_name}, inplace=True)

    if combined_df.empty:

        combined_df = df
    else:

        combined_df = pd.merge(combined_df, df, on=list(df.columns.difference([ndvi_column_name])), how='outer')


output_file = os.path.join(data_dir, 'combined_plant_monthly_ndvi.csv')
combined_df.to_csv(output_file, index=False)

print(f'Combined data saved to {output_file}')


Combined data saved to /content/drive/MyDrive/Plant_planet/combined_plant_monthly_ndvi.csv


In [ ]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076 entries, 0 to 1075
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   system:index  1076 non-null   object 
 1   country       1076 non-null   object 
 2   descriptio    1076 non-null   object 
 3   NDVI_01       1074 non-null   float64
 4   planting_d    873 non-null    object 
 5   project_id    1076 non-null   object 
 6   site_id_re    1076 non-null   object 
 7   site_sqkm     1076 non-null   float64
 8   survival_r    887 non-null    float64
 9   trees_plan    1076 non-null   float64
 10  .geo          1076 non-null   object 
 11  NDVI_02       1074 non-null   float64
 12  NDVI_03       1074 non-null   float64
 13  NDVI_04       1074 non-null   float64
 14  NDVI_05       1074 non-null   float64
 15  NDVI_06       1074 non-null   float64
 16  NDVI_07       1074 non-null   float64
 17  NDVI_08       1074 non-null   float64
 18  NDVI_09       1074 non-null 

In [ ]:
df=combined_df
def find_top_three_ndvi_months(row):
    ndvi_columns = ['NDVI_01', 'NDVI_02', 'NDVI_03', 'NDVI_04', 'NDVI_05', 'NDVI_06',
                    'NDVI_07', 'NDVI_08', 'NDVI_09', 'NDVI_010', 'NDVI_011', 'NDVI_012']
    ndvi_values = []

    # Collecting NDVI values and their corresponding months
    for column in ndvi_columns:
        month_number = column.split('_')[1]
        ndvi_value = row[column]
        ndvi_values.append((ndvi_value, month_number))

    # Sorting the list by NDVI values in descending order
    ndvi_values.sort(reverse=True, key=lambda x: x[0])

    # Extracting the top three months
    top_three_months = [int(month) for _, month in ndvi_values[:3]]

    return top_three_months


df['Top_Three_NDVI_Months'] = df.apply(find_top_three_ndvi_months, axis=1)


ndvi_columns_to_drop = ['NDVI_01', 'NDVI_02', 'NDVI_03', 'NDVI_04', 'NDVI_05', 'NDVI_06',
                        'NDVI_07', 'NDVI_08', 'NDVI_09', 'NDVI_010', 'NDVI_011', 'NDVI_012']


df.drop(columns=ndvi_columns_to_drop, inplace=True)


df.tail()

,system:index,country,descriptio,planting_d,project_id,site_id_re,site_sqkm,survival_r,trees_plan,.geo,Top_Three_NDVI_Months
1071,000000000000000000bc,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""GeometryCollection"",""geometries"":[{""t...","[10, 3, 1]"
1072,000000000000000000bd,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""Polygon"",""coordinates"":[[[38.24284252...","[10, 1, 7]"
1073,000000000000000000be,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""Polygon"",""coordinates"":[[[38.25322336...","[10, 2, 3]"
1074,000000000000000000bf,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""Polygon"",""coordinates"":[[[38.24181244...","[10, 1, 2]"
1075,000000000000000000c0,TZ,ForestNation's mission is to help humanity thr...,NaN,proj_6SfzpmoxRIxOlEwzXu6aWQov,reforest_site_72,3694071.01,100.0,477975.0,"{""type"":""Polygon"",""coordinates"":[[[38.37923753...","[6, 1, 12]"


In [ ]:
df.to_csv("/content/drive/MyDrive/Plant_planet/combined_plant_monthly_ndvi.csv", index=False)

*** Conclusion ***
Manually or through code  combine the unique columns to have complete dataset for each reforestation organization

In [ ]:
import os
import pandas as pd

dataframes = []
all_columns = set()


folder_path = '/content/drive/MyDrive/Plant_planet'

for file_name in os.listdir(folder_path):

    if file_name.endswith('.csv') and not file_name.startswith('monthly_plant_'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        dataframes.append(df)
        all_columns.update(df.columns)


common_columns = list(set.intersection(*[set(df.columns) for df in dataframes]))


merged_df = dataframes[0][common_columns].copy()


unique_columns_first_df = list(set(dataframes[0].columns) - set(common_columns))
merged_df = pd.concat([merged_df, dataframes[0][unique_columns_first_df]], axis=1)


for df in dataframes[1:]:

    unique_columns = list(set(df.columns) - set(common_columns))

    merged_df = pd.concat([merged_df, df[unique_columns]], axis=1)


columns_to_drop = ["system:index", "planting_d", "Unnamed: 0"]
merged_df = merged_df.drop(columns=[col for col in columns_to_drop if col in merged_df.columns])


if 'site_id_re_x' in merged_df.columns and 'site_id_re_y' in merged_df.columns:
    merged_df['site_id_re'] = merged_df['site_id_re_x'].combine_first(merged_df['site_id_re_y'])
    merged_df = merged_df.drop(columns=['site_id_re_x', 'site_id_re_y'])

merged_df['host_name'] = 'plant for plant'


merged_df['url'] = 'https://web.plant-for-the-planet.org/en/sirpnigeria-org?site=site_bKgRAzMysTSivm1'

output_file_path = os.path.join(folder_path, 'plant_planet.csv')
merged_df.to_csv(output_file_path, index=False)

print(f"Merged CSV saved to {output_file_path}")


Merged CSV saved to /content/drive/MyDrive/Plant_planet/plant_planet.csv
